# Import packages

In [ ]:
import pandas as pd
import json
import random
import os
import numpy as np


# Import data

In [2]:
# Load the JSON file
with open("questions.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)

print(type(data))
print(data[0])

<class 'list'>
{'task_id': '8e867cd7-cff9-4e6c-867a-ff5ddc2550be', 'question': 'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.', 'Level': '1', 'file_name': ''}


In [3]:
# Select random question
random_question = random.choice(data)
print(random_question)

{'task_id': 'cabe07ed-9eca-40ea-8ead-410ef5e83f91', 'question': "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?", 'Level': '1', 'file_name': ''}


# Set up LLM with Ollama 
I wanted to use LlamaIndex's HuggingFaceAPI but my pc configurations won't let me use this API

In [10]:
# before running the following code I first installed Ollama () on my pc
# then in the terminal I ran "ollama pull qwen3"/"ollama pull qwen3:0.6b"

from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="qwen3:0.6b", #qwen3:latest
    #request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
    temperature = 0.1,
)

In [11]:
# test to see if everything is working
response = llm.complete('What\'s the capital of France?')

In [12]:
print(response)

<think>
Okay, the user is asking about the capital of France. I know that France's capital is Paris. But wait, let me make sure. I've heard that sometimes people confuse the capital with other countries. For example, in some cases, people might think of the capital of the United States as Washington, D.C. But that's not the case here. France's capital is indeed Paris. I should double-check that. Oh, right, the capital is Paris, and it's a major city in France. No need to mention other countries here. Just confirm that the answer is Paris.
</think>

The capital of France is **Paris**.


I chose qwen because it was finetuned to "think" before answering and I want that for my agent. Of course, this is not ideal for a simple question like the one above.

# Set up Llamaindex (tools, agents, prompt)

In [ ]:
# calculator tool

def calculate(input: dict) -> dict:
    """Simple calculator function"""

    expression = str(input['input'])
    
    # Remove any potentially unsafe operations
    if any(unsafe in expression for unsafe in ["import", "exec", "eval", "compile", "open", "__"]):
        return {"error": "Unsafe expression"}
    
    try:
        # Use a safer approach to evaluate mathematical expressions
        # This is a simplified version - in production you'd want more safeguards
        allowed_symbols = {
            'sqrt': np.sqrt, 'pi': np.pi, 'e': np.e,
            'sin': np.sin, 'cos': np.cos, 'tan': np.tan,
            'log': np.log, 'log10': np.log10, 'exp': np.exp,
            'floor': np.floor, 'ceil': np.ceil, 'abs': abs
        }
        
        # Replace common math operations with Python syntax
        expression = expression.replace('^', '**')
        result = eval(expression, {"__builtins__": {}}, allowed_symbols)
        return {"result": result}
    except Exception as e:
        return {"error": f"Failed to calculate: {str(e)}"}

In [ ]:
# wikipedia tool

from llama_index.readers.wikipedia import WikipediaReader

def get_wikipedia_content(input: dict) -> dict:
    """Fetch content from Wikipedia based on a query."""

    query = str(input['input'])
    
    reader = WikipediaReader()
    documents = reader.load_data(query=query)
    if documents:
        return {'result' :documents[0].text}
    else:
        return {'error:' "No content found."}

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from llama_index import Document

video_id = "Ks-_Mh1QhMc"
transcript = YouTubeTranscriptApi.get_transcript(video_id)
text = "\n".join([t["text"] for t in transcript])
document = Document(text=text)

In [ ]:
from llama_index.core.tools import FunctionTool

calculator_tool = FunctionTool.from_defaults(
    fn=calculate,
    name="calculator",
    description="A simple calculator that performs basic arithmetic operations."
)

wikipedia_tool = FunctionTool.from_defaults(
    fn=get_wikipedia_content,
    name="get_wikipedia_content",
    description="A simple content extractor from Wikipedia based on a query."
)

In [ ]:
tools = [calculator_tool, wikipedia_tool]

# Run agent on question to test it

In [ ]:
# copy and paste from above with slight modifications for tool_descriptions and output
def create_system_prompt(tools):
    """Create a more descriptive system prompt that explains the available tools"""
    tool_descriptions = "\n".join([
        f"- {tool._metadata.name}: {tool._metadata.description}" 
        for tool in tools
    ])
    
    system_prompt = f"""You're a helpful AI assistant with the ability to use tools.
        
You have access to the following tools:
{tool_descriptions}

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Important: If a question requires calculation ALWAYS use the appropriate tool rather than trying to answer from your knowledge.
It doesn't matter if  the answer is straightforward, use tools to ensure accuracy and reliability."""

    return system_prompt

In [70]:
# copy and paste from above with slight modifications for tool_descriptions and output
def create_system_prompt(tools):
    """Create a more descriptive system prompt that explains the available tools"""
    tool_descriptions = "\n".join([
        f"- {tool._metadata.name}: {tool._metadata.description}" 
        for tool in tools
    ])
    
    system_prompt = f"""You are a general AI assistant with the ability to use tools.
        
You have access to the following tools:
{tool_descriptions}

I will ask you a question and to answer it you can use the tools available to you.

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Report your thoughts, and finish YOUR FINAL ANSWER with just the result to the question. 
YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 
If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. 
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.

Important: If a question requires calculation ALWAYS use the appropriate tool rather than trying to answer from your knowledge.
It doesn't matter if  the answer is straightforward, use tools to ensure accuracy and reliability.
"""
    return system_prompt

In [71]:
system_prompt = create_system_prompt(tools)

In [72]:
print(system_prompt)

You are a general AI assistant with the ability to use tools.

You have access to the following tools:
- calculator: A simple calculator that performs basic arithmetic operations.

I will ask you a question and to answer it you can use the tools available to you.

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Report your thoughts, and finish YOUR FINAL ANSWER with just the result to the question. 
YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 
If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and writ

In [73]:
from llama_index.core.agent.workflow import ReActAgent

agent = ReActAgent(
    system_prompt=system_prompt,  # must be string
    tools=[calculator_tool], 
    llm=llm,
    verbose=False
)

In [74]:
extract_prompt = """
You are a general AI assistant that must answer to this question:

{{question}}

Report your thoughts, and finish YOUR FINAL ANSWER with just the result to the question. 
YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 
If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. 
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.

Important: If a question requires calculation use the appropriate tool rather than trying to answer from your knowledge.
"""

In [75]:
'''question = 'What is 10^4?'

extract_prompt = extract_prompt.replace("{{question}}", question)'''

extract_prompt = 'What is 10^4?'
print(extract_prompt)

What is 10^4?


In [76]:
import nest_asyncio
from llama_index.core.memory import Memory

# to keep conversation history
memory = Memory.from_defaults(
    token_limit=35000  # Normally you would set this to be closer to the LLM context window (i.e. 75,000, etc.)
)

# run agent with thinking process printed
nest_asyncio.apply()

async def main():
    handler = agent.run(extract_prompt, memory=memory)

    # così vedo come ragiona il LLM
    async for event in handler.stream_events():
        if hasattr(event, "delta"):
            print(event.delta, end="", flush=True)

    result = await handler
    return result

result = await main()

<think>
Okay, the user is asking what 10^4 is. Let me think. 10 raised to the power of 4 means 10 multiplied by itself four times. So 10*10*10*10. Let me calculate that. 10*10 is 100, then 100*10 is 1000, and 1000*10 is 10,000. So the answer should be 10,000. I don't need any tools here because it's a straightforward arithmetic problem.
</think>

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 10,000

In [44]:
print(result.response.blocks[0].text)

2


In [45]:
type(result.response.blocks[0].text)

str